In [1]:
# Feature Store 
## 



In [27]:
import sagemaker
import boto3
import pandas as pd 
import numpy as np 
from sagemaker.feature_store.feature_group import FeatureGroup

In [28]:
sagemaker_session = sagemaker.Session()
REGION  = sagemaker_session.boto_region_name
BUCKET = sagemaker_session.default_bucket()
print(REGION, "\n", BUCKET)
print(boto3.Session().region_name)

boto_session  = boto3.Session(region_name=REGION)
fs_client = boto_session.client(service_name="sagemaker-featurestore-runtime")


us-east-1 
 sagemaker-us-east-1-205930620783
us-east-1


In [29]:
# Intialize Feature Group objects
claims_feature_group_name = "claims-feature-group-20-06-17-56"
customers_feature_group_name = "customers-feature-group-20-06-17-56"

claims_feature_group = FeatureGroup(name=claims_feature_group_name, sagemaker_session=sagemaker_session)
customers_feature_group = FeatureGroup(name=customers_feature_group_name, sagemaker_session=sagemaker_session)

In [30]:
claims_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:205930620783:feature-group/claims-feature-group-20-06-17-56',
 'FeatureGroupName': 'claims-feature-group-20-06-17-56',
 'RecordIdentifierFeatureName': 'policy_id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'policy_id',
   'FeatureType': 'Integral'},
  {'FeatureName': 'incident_severity', 'FeatureType': 'Fractional'},
  {'FeatureName': 'num_vehicles_involved', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_injuries', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_witnesses', 'FeatureType': 'Integral'},
  {'FeatureName': 'police_report_available', 'FeatureType': 'Fractional'},
  {'FeatureName': 'injury_claim', 'FeatureType': 'Integral'},
  {'FeatureName': 'vehicle_claim', 'FeatureType': 'Integral'},
  {'FeatureName': 'total_claim_amount', 'FeatureType': 'Integral'},
  {'FeatureName': 'incident_month', 'FeatureType': 'Integral'},
  {'FeatureName': 'incident_day', 'FeatureType': 'Integral'},
  {'Fe

In [31]:
claims_feature_group.describe()['OfflineStoreConfig']['DataCatalogConfig']['TableName']

'claims_feature_group_20_06_17_56_1745129970'

In [36]:
claims_query = claims_feature_group.athena_query()
print(claims_query.table_name, "\n", claims_query.database)
customers_query = customers_feature_group.athena_query()
print(customers_query.table_name, "\n", customers_query.database)


claims_feature_group_20_06_17_56_1745129970 
 sagemaker_featurestore
customers_feature_group_20_06_17_56_1745129968 
 sagemaker_featurestore


# Write a SQL query to join both the tables and select all the columns

In [47]:
query = f""" 
    SELECT * 
    FROM {claims_query.table_name} LEFT JOIN {customers_query.table_name} 
    ON {claims_query.table_name}.policy_id = {customers_query.table_name}.policy_id 
"""
print(query)

claims_query.run(query, output_location=f"s3://{BUCKET}/FraudDetection_AutoInsurance/data/query_results")
claims_query.wait() 
dataset = claims_query.as_dataframe()

 
    SELECT * 
    FROM claims_feature_group_20_06_17_56_1745129970 LEFT JOIN customers_feature_group_20_06_17_56_1745129968 
    ON claims_feature_group_20_06_17_56_1745129970.policy_id = customers_feature_group_20_06_17_56_1745129968.policy_id 



[04/20/25 10:02:02] INFO     Query db52336d-53cd-4034-ad58-76eda6ac0df2 is being executed.          ]8;id=904799;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=542042;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#6663\6663]8;;\

[04/20/25 10:02:07] INFO     Query db52336d-53cd-4034-ad58-76eda6ac0df2 successfully executed.      ]8;id=341393;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=548169;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#6672\6672]8;;\

In [48]:
dataset_col_order = ['fraud']  + list(dataset.drop(["fraud", "policy_id"], axis=1).columns)
train = dataset.sample(frac=.80, random_state=0)[dataset_col_order]
test = dataset.drop(train.index)[dataset_col_order]
train.to_csv("data/train.csv", index=False)
test.to_csv("data/test.csv", index=False)
dataset.to_csv("data/dataset.csv", index=True)

In [50]:
# Upload the data to S3
s3_client = boto_session.client("s3")
s3_client.upload_file('data/train.csv', Bucket=BUCKET, Key="FraudDetection_AutoInsurance/data/train.csv")
s3_client.upload_file('data/test.csv', Bucket=BUCKET, Key="FraudDetection_AutoInsurance/data/test.csv")
s3_client.upload_file('data/dataset.csv', Bucket=BUCKET, Key="FraudDetection_AutoInsurance/data/dataset.csv")
